# Finetuning NbAILab/nb-bert-large
using Optuna and gender labeled data

bla bla bla noget forklaring


## GPU & Installing packages:

In [ ]:
# Initialize GPU
!nvidia-smi

Mon Dec 18 10:36:57 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              49W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
#### Install packages:
!pip install -q transformers transformers-interpret datasets evaluate spacy spacy_langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 998.1/998.1 kB 55.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### Importing packages:
import pandas as pd
import torch
from datasets import Dataset, DatasetDict, load_metric
from transformers import AutoTokenizer, AutoModelForPreTraining, Trainer, TrainingArguments, AutoModelForSequenceClassification
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import EarlyStoppingCallback

In [ ]:
### loading data:
train_path = "/content/drive/MyDrive/Colab_Notebooks/NLP_EX_COLAB/nb_large_model/train.csv"
df_train = pd.read_csv(train_path)

test_path = "/content/drive/MyDrive/Colab_Notebooks/NLP_EX_COLAB/nb_large_model/test.csv"
df_test = pd.read_csv(test_path)

val_path = "/content/drive/MyDrive/Colab_Notebooks/NLP_EX_COLAB/nb_large_model/valid.csv"
df_val = pd.read_csv(val_path)

train_dataset = Dataset.from_pandas(df_train)
valid_dataset = Dataset.from_pandas(df_val)
test_dataset = Dataset.from_pandas(df_test)

data_set = DatasetDict({
    'train': train_dataset,
    'validation': valid_dataset,
    'test': test_dataset
    })

In [ ]:
!ls # make sure that there isn't a local model folder

drive  sample_data


In [ ]:
# Loading model and tokenizer
model_ckpt = "NbAiLab/nb-bert-large"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
num_labels=2
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at NbAiLab/nb-bert-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [ ]:
### tokenizing data:
dat_tokenized = data_set.map(tokenize, batched=True)

Map:   0%|          | 0/919 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/153 [00:00<?, ? examples/s]

Map:   0%|          | 0/461 [00:00<?, ? examples/s]

## Creating evaluation metrics

In [ ]:
import numpy as np
import evaluate

def compute_metrics(eval_pred):
    metric0 = evaluate.load("accuracy")
    metric1 = evaluate.load("precision")
    metric2 = evaluate.load("recall")
    metric3 = evaluate.load("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric0.compute(predictions=predictions, references=labels)["accuracy"]
    precision = metric1.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = metric2.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1 = metric3.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

### early stopping

In [ ]:
# Makes the model stop when validation loss hasn't improved for n(early_stopping_patience) epochs
early_stop = EarlyStoppingCallback(early_stopping_patience = 10)

### defining hyperparameters
these are the initializing paramaters - vi starter med dem for at få trainer til at køre. Derefter optimerer vi.

Rationalet bag de høje epochs er, at når vi alligevel bruger early_stop stopper den når der ikke er forbedring.

disse bliver overwritet i tuningen


In [ ]:
batch_size = 16 # the gpu limits us to only 16 - else not enough memory
epochs = 200
learning_rate = 2e-5

## Hyperparameter tuning
hertil bruges Optuna

In [ ]:
!pip install -q optuna ray[tune]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 12.1 MB/s eta 0:00:00


**Setting training parameters**

In [ ]:
#!pip uninstall transformers[torch]

In [ ]:
#!pip install transformers[torch]

In [ ]:
training_args = TrainingArguments(output_dir=model_ckpt,
                                  evaluation_strategy = "epoch",
                                  save_strategy = "epoch",
                                  logging_strategy = "epoch",
                                  logging_steps = 50,
                                  num_train_epochs = epochs,
                                  per_device_train_batch_size = batch_size,
                                  per_device_eval_batch_size = batch_size,
                                  learning_rate = learning_rate,
                                  weight_decay=0.01,
                                  load_best_model_at_end=True,
                                  report_to="none",
                                  save_total_limit = 2)

In [ ]:
def model_init():
    return model #the model defined earlier

In [ ]:
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=dat_tokenized["train"],
    eval_dataset=dat_tokenized["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [early_stop]
)

**Setting the hyperparameter space to sample from in the optimization process**

maybe add somekind of argument for the choices made here

In [ ]:
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "weight_decay": trial.suggest_loguniform('weight_decay', 1e-4, 1e-2)
    }

#### **Initializing hyperparameter tuning**

In [ ]:
import optuna
from optuna.samplers import TPESampler


sampler = optuna.samplers.TPESampler()
pruner = optuna.pruners.MedianPruner(n_warmup_steps=10)


best_run = trainer.hyperparameter_search(
    n_trials=10,
    direction="minimize",
    hp_space=hp_space,
    backend = "optuna",
    sampler = sampler,
    pruner = pruner
    )

print(best_run)

[I 2023-12-18 10:40:30,529] A new study created in memory with name: no-name-056f7bde-c5d5-4f2d-9c3b-58dab7f72206
<ipython-input-13-d658364f193f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "weight_decay": trial.suggest_loguniform('weight_decay', 1e-4, 1e-2)
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.619200,0.683212,0.620915,0.609387,0.620915,0.614793
2,0.616000,0.598020,0.725490,0.526336,0.725490,0.610071
3,0.602500,0.594861,0.725490,0.526336,0.725490,0.610071
4,0.556400,0.494924,0.771242,0.763628,0.771242,0.766522
5,0.412500,0.403909,0.830065,0.827247,0.830065,0.815785
6,0.399500,0.420258,0.843137,0.848651,0.843137,0.845263
7,0.351600,0.456707,0.843137,0.838929,0.843137,0.834087
8,0.259600,0.504348,0.843137,0.837865,0.843137,0.837609
9,0.199200,0.438926,0.862745,0.861793,0.862745,0.862226
10,0.173200,0.624236,0.862745,0.859342,0.862745,0.859913


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023-12-18 11:01:10,144] Trial 0 finished with value: 3.335914280888093 and parameters: {'learning_rate': 3.1650041323741484e-05, 'weight_decay': 0.00032048150087580785}. Best is trial 0 with value: 3.335914280888093.
<ipython-input-13-d658364f193f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/rele

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.286900,0.405207,0.856209,0.852973,0.856209,0.853887
2,0.197000,0.492453,0.856209,0.852597,0.856209,0.849592
3,0.138600,0.632004,0.823529,0.827887,0.823529,0.825363
4,0.126300,0.521416,0.875817,0.877991,0.875817,0.867883
5,0.122200,0.452794,0.882353,0.879839,0.882353,0.879376
6,0.100500,0.575418,0.875817,0.873015,0.875817,0.873255
7,0.076100,0.639285,0.869281,0.866505,0.869281,0.867170
8,0.078200,0.683926,0.862745,0.860284,0.862745,0.861118
9,0.063500,0.699345,0.869281,0.867612,0.869281,0.868270
10,0.060700,0.617941,0.875817,0.873767,0.875817,0.870785


[I 2023-12-18 11:16:13,287] Trial 1 finished with value: 3.4722371058276633 and parameters: {'learning_rate': 1.55277936352044e-05, 'weight_decay': 0.0002665993561047042}. Best is trial 0 with value: 3.335914280888093.
<ipython-input-13-d658364f193f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "weight_decay": trial.suggest_loguniform('weight_decay', 1e-4, 1e-2)


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.168100,0.502686,0.862745,0.863823,0.862745,0.863242


[I 2023-12-18 12:31:06,858] Trial 8 pruned. 
<ipython-input-13-d658364f193f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "weight_decay": trial.suggest_loguniform('weight_decay', 1e-4, 1e-2)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.190300,0.690570,0.745098,0.796732,0.745098,0.757272


[I 2023-12-18 12:32:18,276] Trial 9 pruned. 


BestRun(run_id='2', objective=2.587387718010555, hyperparameters={'learning_rate': 7.242436679594794e-05, 'weight_decay': 0.00022334799548955845}, run_summary=None)


In [ ]:
#BestRun(run_id='2', objective=2.587387718010555, hyperparameters={'learning_rate': 7.242436679594794e-05, 'weight_decay': 0.00022334799548955845}, run_summary=None)


# calling best run
print(best_run)

# Create a DataFrame from the dictionary
df = pd.DataFrame(best_run)

# Define the file path where the CSV will be saved
csv_file_path = '/content/drive/MyDrive/Colab_Notebooks/NLP_EX_COLAB/nb_large_model/best_run_nbl.csv'  # You can specify a different path here

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

BestRun(run_id='2', objective=2.587387718010555, hyperparameters={'learning_rate': 7.242436679594794e-05, 'weight_decay': 0.00022334799548955845}, run_summary=None)
